<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week06/pyspark/Pyspark_join_multiple_dstreams_json2dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

In [2]:
%%capture
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json

In [5]:
kafka_topic_name = "my-first-topic"
kafka_topic_name2 = "my-first-topic2"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [21]:
sc = pyspark.SparkContext()
spark = SparkSession(sc)
ssc = StreamingContext(sc,2) # Show results every 2 seconds

Ex1

In [15]:
def readMyStream(rdd):
  if not rdd.isEmpty():
    df = spark.read.json(rdd)
    print(df.dtypes)
    print('Started the Process')
    print('Selection of Columns')
    df = df.select('firstname','lastname')
    df.show()

Ex2

In [22]:
def readMyStream2(rdd):
  if not rdd.isEmpty():
    df = spark.read.json(rdd)

    # The inferred schema can be visualized using the printSchema() method
    df.printSchema()

    # Creates a temporary view using the DataFrame
    df.createOrReplaceTempView("people")

    # SQL statements can be run by using the sql methods provided by spark
    resultDF = spark.sql("SELECT SUM(salary) FROM people")
    resultDF.show()

In [ ]:
kvs1 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'smallest'
                        })

kvs2 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name2], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'
                        })

kvs2.pprint()
parsed = kvs2.map(lambda x: x[1])
parsed.foreachRDD( lambda rdd: readMyStream2(rdd) )


ssc.start()
ssc.awaitTerminationOrTimeout(30)
ssc.stop()
sc.stop()

In [18]:
ssc.stop()
sc.stop()